In [8]:
import optuna
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn import naive_bayes
from sklearn import ensemble, tree
import xgboost as xgb
import catboost
from sklearn import linear_model
from sklearn import model_selection
from sklearn import metrics
from functools import partial
from skopt import space, gp_minimize
from hyperopt import hp, fmin, tpe, Trials
from hyperopt.pyll.base import scope
from sklearn import metrics
import warnings
warnings.filterwarnings('ignore')

# Load dataset and preproccessing.

In [9]:
def remove_high_correlation(dataset, threshold):
    col_names = []
    for i in range(dataset.corr().shape[1] - 1):
        for j in range(i):
            if abs(dataset.corr().iloc[i,j]) >= threshold:
                col_names.append(dataset.corr().columns[i])
    return col_names

data_sets_path = '/home/hamza/Downloads/DataSets/All_Classification_Models/'
df = pd.concat([pd.read_csv(data_sets_path + 'churn-bigml-80.csv'), pd.read_csv(data_sets_path + 'churn-bigml-20.csv')])

for i in df.select_dtypes(include = [object, bool]).columns:
    df[i] = LabelEncoder().fit_transform(df[i])

df.drop(remove_high_correlation(df, 0.9), axis = 1, inplace = True)

X = df.drop('Churn', axis = 1).values
y = df['Churn'].values


# Support Vector Machine with different hyper parameter tuning techniques.

In [ ]:
# apply (Support Vector Machine) with GridSearch.

In [6]:
algo = SVC()

model = model_selection.GridSearchCV(algo, param_grid = {
    'degree' : [i for i in range(1,5)],
    'kernel' : ['rbf','poly','sigmoid'],
}, scoring = 'accuracy', cv = model_selection.StratifiedShuffleSplit(n_splits = 10))
model.fit(X,y)
print(model.best_params_)
print(model.best_score_)

{'degree': 4, 'kernel': 'poly'}
0.8565868263473053


In [ ]:
# apply (Support Vector Machine) with RandomSearch.

In [8]:
model = model_selection.RandomizedSearchCV(algo, param_distributions = {
    'degree' : np.arange(1,5),
    'kernel' : ['poly','rbf','sigmoid']
},
scoring = 'accuracy', cv = model_selection.StratifiedShuffleSplit(n_splits = 10), n_iter = 15)

model.fit(X,y)
print(model.best_params_)
print(model.best_score_)

{'kernel': 'poly', 'degree': 1}
0.8562874251497006


In [ ]:
# apply (Support Vector Machine) with gp minimize.

In [9]:
def optimize(params, params_names, x, y):
    params = dict(zip(params_names, params))
    model = SVC(**params)
    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return -1.0 * np.mean(accuracies)

param_spaces = [
    space.Integer(1,5, name = 'degree'),
    space.Categorical(['poly','rbf','sigmoid'], name = 'kernel')
]

param_names = ['degree','kernel']

optimaization_function = partial(optimize, params_names = param_names, x = X, y = y)

result = gp_minimize(optimaization_function,dimensions = param_spaces, n_calls = 15, n_random_starts = 10, verbose = 10)
print(dict(zip(param_names, result.x)))

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.7018
Function value obtained: -0.8551
Current minimum: -0.8551
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.8083
Function value obtained: -0.8488
Current minimum: -0.8551
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.8039
Function value obtained: -0.8488
Current minimum: -0.8551
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.8133
Function value obtained: -0.8488
Current minimum: -0.8551
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 1.1287
Function value obtained: -0.8611
Current minimum: -0.8611
Iteration No: 6 started. 

In [ ]:
# apply (Support Vector Machine) with hyperopt.

In [10]:
def optimize(params, x, y):
    model = SVC(**params)
    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return -1.0 * np.mean(accuracies)

param_spaces = {
    'degree': scope.int(hp.quniform('degree',1,5,1)),
    'kernel': hp.choice('kernel',['poly','rbf','sigmoid'])
}

optimaization_function = partial(optimize,x = X, y = y)

result = fmin(fn = optimaization_function,space = param_spaces, max_evals = 15, trials = Trials(), algo = tpe.suggest)
print(result)

100%|██████████| 15/15 [00:11<00:00,  1.29trial/s, best loss: -0.8550859705282494]
{'degree': 2.0, 'kernel': 0}


In [ ]:
# apply (Support Vector Machine) with optuna.

In [11]:
def optimize(trial, x, y):
    kernel = trial.suggest_categorical('kernel', ['poly','rbf','sigmoid'])
    degree = trial.suggest_int('degree', 1, 5)
    model = SVC(
        kernel = kernel,
        degree = degree
    )

    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return np.mean(accuracies)

optimaization_function = partial(optimize,x = X, y = y)

study = optuna.create_study(direction = 'maximize')
study.optimize(optimaization_function, n_trials = 15)
print(study.best_params, study.best_value)

[I 2022-08-10 15:35:10,178] A new study created in memory with name: no-name-926440b0-8282-4049-87c8-9a7e91002740
[I 2022-08-10 15:35:10,699] Trial 0 finished with value: 0.8550859705282494 and parameters: {'kernel': 'poly', 'degree': 1}. Best is trial 0 with value: 0.8550859705282494.
[I 2022-08-10 15:35:11,536] Trial 1 finished with value: 0.854786120453287 and parameters: {'kernel': 'poly', 'degree': 4}. Best is trial 0 with value: 0.8550859705282494.
[I 2022-08-10 15:35:12,348] Trial 2 finished with value: 0.8487850669259964 and parameters: {'kernel': 'sigmoid', 'degree': 3}. Best is trial 0 with value: 0.8550859705282494.
[I 2022-08-10 15:35:13,064] Trial 3 finished with value: 0.8550859705282494 and parameters: {'kernel': 'rbf', 'degree': 2}. Best is trial 0 with value: 0.8550859705282494.
[I 2022-08-10 15:35:13,869] Trial 4 finished with value: 0.8487850669259964 and parameters: {'kernel': 'sigmoid', 'degree': 1}. Best is trial 0 with value: 0.8550859705282494.
[I 2022-08-10 15:

{'kernel': 'poly', 'degree': 1} 0.8550859705282494


# K Nearest Neighbor with different hyper parameter tuning techniques.

In [ ]:
# apply K Nearest Neighbor with GridSearch.

In [30]:
algo = KNeighborsClassifier()

model = model_selection.GridSearchCV(algo, param_grid = {
    'n_neighbors' : [i for i in range(5,101)],
    'algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'metric' : ['euclidean', 'manhattan']
}, scoring = 'accuracy', cv = model_selection.StratifiedShuffleSplit(n_splits = 10))
model.fit(X,y)
print(model.best_params_)
print(model.best_score_)

{'algorithm': 'auto', 'metric': 'manhattan', 'n_neighbors': 5}
0.8736526946107785


In [ ]:
# apply K Nearest Neighbor with RandomeSearch.

In [29]:
model = model_selection.RandomizedSearchCV(algo, param_distributions = {
    'n_neighbors' : np.arange(5,100),
    'algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'metric' : ['euclidean', 'manhattan']
},
scoring = 'accuracy', cv = model_selection.StratifiedShuffleSplit(n_splits = 10), n_iter = 15)

model.fit(X,y)
print(model.best_params_)
print(model.best_score_)

{'n_neighbors': 43, 'metric': 'euclidean', 'algorithm': 'auto'}
0.8649700598802393


In [ ]:
# apply K Nearest Neighbor with gp minimize.

In [26]:
def optimize(params, params_names, x, y):
    params = dict(zip(params_names, params))
    model = KNeighborsClassifier(**params)
    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return -1.0 * np.mean(accuracies)

param_spaces = [
    space.Integer(5,100, name = 'n_neighbors'),
    space.Categorical(['auto', 'ball_tree', 'kd_tree', 'brute'], name = 'algorithm')
    
]

param_names = ['n_neighbors','algorithm']

optimaization_function = partial(optimize, params_names = param_names, x = X, y = y)

result = gp_minimize(optimaization_function,dimensions = param_spaces, n_calls = 15, n_random_starts = 10, verbose = 10)
print(dict(zip(param_names, result.x)))

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.2930
Function value obtained: -0.8644
Current minimum: -0.8644
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.2252
Function value obtained: -0.8659
Current minimum: -0.8659
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.2436
Function value obtained: -0.8629
Current minimum: -0.8659
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.2123
Function value obtained: -0.8644
Current minimum: -0.8659
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 0.2262
Function value obtained: -0.8671
Current minimum: -0.8671
Iteration No: 6 started. 

In [ ]:
# apply K Nearest Neighbor with hyperopt.

In [17]:
def optimize(params, x, y):
    model = KNeighborsClassifier(**params)
    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return -1.0 * np.mean(accuracies)

param_spaces = {
    'n_neighbors': scope.int(hp.quniform('n_neighbors',5,100,1)),
    'algorithm': hp.choice('algorithm',['auto', 'ball_tree', 'kd_tree', 'brute']),
    'metric': hp.choice('metric',['euclidean', 'manhattan'])
}

optimaization_function = partial(optimize,x = X, y = y)

result = fmin(fn = optimaization_function,space = param_spaces, max_evals = 15, trials = Trials(), algo = tpe.suggest)
print(result)

100%|██████████| 15/15 [00:04<00:00,  3.69trial/s, best loss: -0.8730891311101207]
{'algorithm': 2, 'metric': 0, 'n_neighbors': 9.0}


In [ ]:
# apply K Nearest Neighbor with optuna.

In [18]:
def optimize(trial, x, y):
    n_neighbors = trial.suggest_int('n_neighbors', 5,100)
    algorithm = trial.suggest_categorical('algorithm', ['auto', 'ball_tree', 'kd_tree', 'brute'])
    metric = trial.suggest_categorical('metric', ['euclidean', 'manhattan'])
    model = KNeighborsClassifier(
        n_neighbors = n_neighbors,
        algorithm = algorithm,
        metric = metric
    )

    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return np.mean(accuracies)

optimaization_function = partial(optimize,x = X, y = y)

study = optuna.create_study(direction = 'maximize')
study.optimize(optimaization_function, n_trials = 15)
print(study.best_params, study.best_value)

[I 2022-08-10 16:08:50,704] A new study created in memory with name: no-name-0faf6448-d439-4583-9006-26c11f6e5f00
[I 2022-08-10 16:08:50,933] Trial 0 finished with value: 0.8682897290093692 and parameters: {'n_neighbors': 29, 'algorithm': 'auto', 'metric': 'euclidean'}. Best is trial 0 with value: 0.8682897290093692.
[I 2022-08-10 16:08:51,244] Trial 1 finished with value: 0.8604877741309526 and parameters: {'n_neighbors': 93, 'algorithm': 'ball_tree', 'metric': 'euclidean'}. Best is trial 0 with value: 0.8682897290093692.
[I 2022-08-10 16:08:51,575] Trial 2 finished with value: 0.8604882243562904 and parameters: {'n_neighbors': 43, 'algorithm': 'auto', 'metric': 'manhattan'}. Best is trial 0 with value: 0.8682897290093692.
[I 2022-08-10 16:08:51,732] Trial 3 finished with value: 0.8640886763825295 and parameters: {'n_neighbors': 55, 'algorithm': 'brute', 'metric': 'euclidean'}. Best is trial 0 with value: 0.8682897290093692.
[I 2022-08-10 16:08:52,053] Trial 4 finished with value: 0.8

{'n_neighbors': 7, 'algorithm': 'auto', 'metric': 'euclidean'} 0.869788529158844


# Naive Bayes with different hyper parameter tuning techniques.

In [ ]:
# apply Naive Bayes with GridSearch.

In [33]:
algo = naive_bayes.BernoulliNB()

model = model_selection.GridSearchCV(algo, param_grid = {
    'alpha' : [i for i in range(1,11)],
}, scoring = 'accuracy', cv = model_selection.StratifiedShuffleSplit(n_splits = 10))
model.fit(X,y)
print('BernoulliNB: ')
print(model.best_params_)
print(model.best_score_)
#########################
algo = naive_bayes.MultinomialNB()

model = model_selection.GridSearchCV(algo, param_grid = {
    'alpha' : [i for i in range(1,11)],
}, scoring = 'accuracy', cv = model_selection.StratifiedShuffleSplit(n_splits = 10))
model.fit(X,y)
print('MultinomialNB: ')
print(model.best_params_)
print(model.best_score_)
##########################
algo = naive_bayes.GaussianNB()

model = model_selection.GridSearchCV(algo, param_grid = {
}, scoring = 'accuracy', cv = model_selection.StratifiedShuffleSplit(n_splits = 10))
model.fit(X,y)
print('GaussianNB: ')
print(model.best_params_)
print(model.best_score_)

BernoulliNB: 
{'alpha': 1}
0.8553892215568861
MultinomialNB: 
{'alpha': 1}
0.6802395209580838
GaussianNB: 
{}
0.8517964071856288


In [ ]:
# apply Naive Bayes with RandomSearch.

In [37]:
algo = naive_bayes.BernoulliNB()

model = model_selection.RandomizedSearchCV(algo, param_distributions = {
    'alpha' : [i for i in range(1,11)],
}, scoring = 'accuracy', cv = model_selection.StratifiedShuffleSplit(n_splits = 10))
model.fit(X,y)
print('BernoulliNB: ')
print(model.best_params_)
print(model.best_score_)
#########################
algo = naive_bayes.MultinomialNB()

model = model_selection.RandomizedSearchCV(algo, param_distributions = {
    'alpha' : np.arange(1,11)
}, scoring = 'accuracy', cv = model_selection.StratifiedShuffleSplit(n_splits = 10))
model.fit(X,y)
print('MultinomialNB: ')
print(model.best_params_)
print(model.best_score_)
##########################
algo = naive_bayes.GaussianNB()

model = model_selection.RandomizedSearchCV(algo, param_distributions = {
}, scoring = 'accuracy', cv = model_selection.StratifiedShuffleSplit(n_splits = 10))
model.fit(X,y)
print('GaussianNB: ')
print(model.best_params_)
print(model.best_score_)

BernoulliNB: 
{'alpha': 1}
0.8559880239520957
MultinomialNB: 
{'alpha': 2}
0.6964071856287426
GaussianNB: 
{}
0.8514970059880239


In [ ]:
# apply Naive Bayes with gp minimize.

In [46]:
def optimize(params, params_names, x, y):
    params = dict(zip(params_names, params))
    model = naive_bayes.BernoulliNB(**params)
    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return -1.0 * np.mean(accuracies)

param_spaces = [
    space.Integer(1,10, name = 'alpha')
    
]

param_names = ['alpha']

optimaization_function = partial(optimize, params_names = param_names, x = X, y = y)

result = gp_minimize(optimaization_function,dimensions = param_spaces, n_calls = 10, n_random_starts = 10, verbose = 10)
print('BernoulliNB:')
print(dict(zip(param_names, result.x)))
print('###############################################################################')
def optimize(params, params_names, x, y):
    params = dict(zip(params_names, params))
    model = naive_bayes.MultinomialNB(**params)
    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return -1.0 * np.mean(accuracies)

param_spaces = [
    space.Integer(1,10, name = 'alpha')
    
]

param_names = ['alpha']

optimaization_function = partial(optimize, params_names = param_names, x = X, y = y)

result = gp_minimize(optimaization_function,dimensions = param_spaces, n_calls = 10, n_random_starts = 10, verbose = 10)
print('MultinomialNB:')
print(dict(zip(param_names, result.x)))


Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.0132
Function value obtained: -0.8548
Current minimum: -0.8548
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.0124
Function value obtained: -0.8548
Current minimum: -0.8548
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.0123
Function value obtained: -0.8548
Current minimum: -0.8548
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.0112
Function value obtained: -0.8548
Current minimum: -0.8548
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 0.0094
Function value obtained: -0.8548
Current minimum: -0.8548
Iteration No: 6 started. 

In [ ]:
# apply Naive Bayes with hyperopt.

In [49]:
def optimize(params, x, y):
    model = naive_bayes.BernoulliNB(**params)
    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return -1.0 * np.mean(accuracies)

param_spaces = {
    'alpha': scope.int(hp.quniform('alpha',1,10,1)),
}

optimaization_function = partial(optimize,x = X, y = y)

result = fmin(fn = optimaization_function,space = param_spaces, max_evals = 15, trials = Trials(), algo = tpe.suggest)
print('BernoulliNB: ')
print(result)
print('##################################################################################################')
def optimize(params, x, y):
    model = naive_bayes.MultinomialNB(**params)
    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return -1.0 * np.mean(accuracies)

param_spaces = {
    'alpha': scope.int(hp.quniform('alpha',1,10,1)),
}

optimaization_function = partial(optimize,x = X, y = y)

result = fmin(fn = optimaization_function,space = param_spaces, max_evals = 15, trials = Trials(), algo = tpe.suggest)
print('MultinomialNB')
print(result)

100%|██████████| 15/15 [00:00<00:00, 94.10trial/s, best loss: -0.854785670227949]
BernoulliNB: 
{'alpha': 8.0}
##################################################################################################
100%|██████████| 15/15 [00:00<00:00, 129.40trial/s, best loss: -0.6852771812292052]
MultinomialNB
{'alpha': 7.0}


In [ ]:
# apply Naive Bayes with optuna.

In [51]:
def optimize(trial, x, y):
    alpha = trial.suggest_int('alpha', 1,10)
    model = naive_bayes.BernoulliNB(
        alpha = alpha
    )

    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return np.mean(accuracies)

optimaization_function = partial(optimize,x = X, y = y)

study = optuna.create_study(direction = 'maximize')
study.optimize(optimaization_function, n_trials = 10)
print('BernoulliNB:')
print(study.best_params, study.best_value)
print('############################################################################################')
def optimize(trial, x, y):
    alpha = trial.suggest_int('alpha', 1,10)
    model = naive_bayes.MultinomialNB(
        alpha = alpha
    )

    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return np.mean(accuracies)

optimaization_function = partial(optimize,x = X, y = y)

study = optuna.create_study(direction = 'maximize')
study.optimize(optimaization_function, n_trials = 10)
print('MultinomialNB:')
print(study.best_params, study.best_value)


[I 2022-08-10 17:25:59,778] A new study created in memory with name: no-name-46474925-3a07-4e77-b88e-18f9629695f6
[I 2022-08-10 17:25:59,792] Trial 0 finished with value: 0.854785670227949 and parameters: {'alpha': 8}. Best is trial 0 with value: 0.854785670227949.
[I 2022-08-10 17:25:59,807] Trial 1 finished with value: 0.854785670227949 and parameters: {'alpha': 9}. Best is trial 0 with value: 0.854785670227949.
[I 2022-08-10 17:25:59,821] Trial 2 finished with value: 0.854785670227949 and parameters: {'alpha': 7}. Best is trial 0 with value: 0.854785670227949.
[I 2022-08-10 17:25:59,835] Trial 3 finished with value: 0.854785670227949 and parameters: {'alpha': 8}. Best is trial 0 with value: 0.854785670227949.
[I 2022-08-10 17:25:59,850] Trial 4 finished with value: 0.8544858201529866 and parameters: {'alpha': 10}. Best is trial 0 with value: 0.854785670227949.
[I 2022-08-10 17:25:59,864] Trial 5 finished with value: 0.854785670227949 and parameters: {'alpha': 3}. Best is trial 0 wit

BernoulliNB:
{'alpha': 8} 0.854785670227949
############################################################################################
MultinomialNB:
{'alpha': 9} 0.6852771812292052


# Decision Tree Classifier with different hyper parameter tuning techniques.

In [ ]:
# apply Decision Tree Classifier with GridSearch.

In [9]:
algo = tree.DecisionTreeClassifier()

model = model_selection.GridSearchCV(algo, param_grid = {
    'criterion': ['gini','entropy','log_loss'],
    'splitter': ['best','random'],
    'max_depth': [i for i in range(8,33)],
    "min_samples_leaf": [i for i in range(2,9)]
}, scoring = 'accuracy', cv = model_selection.StratifiedShuffleSplit(n_splits = 10))
model.fit(X,y)
print(model.best_params_)
print(model.best_score_)

{'criterion': 'gini', 'max_depth': 8, 'min_samples_leaf': 3, 'splitter': 'best'}
0.945808383233533


In [ ]:
# apply Decision Tree Classifier with RandomizedSearch.

In [3]:
algo = tree.DecisionTreeClassifier()
model = model_selection.RandomizedSearchCV(algo, param_distributions = {
    'criterion': ['gini','entropy','log_loss'],
    'splitter': ['best','random'],
    'max_depth': np.arange(8,33),
    "min_samples_leaf": np.arange(2,9)
},
scoring = 'accuracy', cv = model_selection.StratifiedShuffleSplit(n_splits = 10), n_iter = 15)

model.fit(X,y)
print(model.best_params_)
print(model.best_score_)

{'splitter': 'best', 'min_samples_leaf': 5, 'max_depth': 8, 'criterion': 'entropy'}
0.9419161676646708


In [ ]:
# apply Decision Tree Classifier with gp minimize.

In [4]:
def optimize(params, params_names, x, y):
    params = dict(zip(params_names, params))
    model = tree.DecisionTreeClassifier(**params)
    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return -1.0 * np.mean(accuracies)

param_spaces = [
    space.Categorical(['gini','entropy','log_loss'], name = 'criterion'),
    space.Categorical(['best','random'], name = 'splitter'),
    space.Integer(8,33, name = 'max_depth'),
    space.Integer(2,9, name = 'min_samples_leaf')
]

param_names = ['criterion','splitter','max_depth','min_samples_leaf']

optimaization_function = partial(optimize, params_names = param_names, x = X, y = y)

result = gp_minimize(optimaization_function,dimensions = param_spaces, n_calls = 15, n_random_starts = 10, verbose = 10)
print(dict(zip(param_names, result.x)))

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.1172
Function value obtained: -0.9328
Current minimum: -0.9328
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.0630
Function value obtained: -0.9385
Current minimum: -0.9385
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.0173
Function value obtained: -0.9190
Current minimum: -0.9385
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.0225
Function value obtained: -0.9175
Current minimum: -0.9385
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 0.1060
Function value obtained: -0.9232
Current minimum: -0.9385
Iteration No: 6 started. 

In [ ]:
# apply Decision Tree Classifier with hyperopt.

In [5]:
def optimize(params, x, y):
    model = tree.DecisionTreeClassifier(**params)
    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return -1.0 * np.mean(accuracies)

param_spaces = {
    'criterion': hp.choice('criterion',['gini','entropy','log_loss']),
    'splitter': hp.choice('splitter',['best','random']),
    'max_depth': scope.int(hp.quniform('max_depth',8,33,1)),
    'min_samples_leaf': scope.int(hp.quniform('min_samples_leaf',2,9,1))
}

optimaization_function = partial(optimize,x = X, y = y)

result = fmin(fn = optimaization_function,space = param_spaces, max_evals = 15, trials = Trials(), algo = tpe.suggest)
print(result)

100%|██████████| 15/15 [00:00<00:00, 18.23trial/s, best loss: -0.9399962181071626]
{'criterion': 2, 'max_depth': 9.0, 'min_samples_leaf': 4.0, 'splitter': 0}


In [ ]:
# apply Decision Tree Classifier with optuna.

In [6]:
def optimize(trial, x, y):
    criterion = trial.suggest_categorical('criterion', ['gini','entropy','log_loss'])
    splitter = trial.suggest_categorical('splitter', ['best','random'])
    max_depth = trial.suggest_int('max_depth', 8, 32)
    min_samples_leaf = trial.suggest_float('min_samples_leaf',2,8)
    model = tree.DecisionTreeClassifier(
        criterion = criterion,
        splitter = splitter,
        max_depth = max_depth,
        min_samples_leaf = min_samples_leaf
    )

    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return np.mean(accuracies)

optimaization_function = partial(optimize,x = X, y = y)

study = optuna.create_study(direction = 'maximize')
study.optimize(optimaization_function, n_trials = 15)
print(study.best_params, study.best_value)

[I 2022-08-12 09:44:30,647] A new study created in memory with name: no-name-043ce372-d6fb-4b19-8e32-b84c3b25e637
[I 2022-08-12 09:44:30,658] Trial 0 finished with value: 0.8550859705282494 and parameters: {'criterion': 'gini', 'splitter': 'random', 'max_depth': 27, 'min_samples_leaf': 2.9380319373672625}. Best is trial 0 with value: 0.8550859705282494.
[I 2022-08-12 09:44:30,666] Trial 1 finished with value: 0.8550859705282494 and parameters: {'criterion': 'entropy', 'splitter': 'random', 'max_depth': 23, 'min_samples_leaf': 5.054653759916127}. Best is trial 0 with value: 0.8550859705282494.
[I 2022-08-12 09:44:30,674] Trial 2 finished with value: 0.8550859705282494 and parameters: {'criterion': 'log_loss', 'splitter': 'random', 'max_depth': 24, 'min_samples_leaf': 5.615240431204665}. Best is trial 0 with value: 0.8550859705282494.
[I 2022-08-12 09:44:30,683] Trial 3 finished with value: 0.8550859705282494 and parameters: {'criterion': 'log_loss', 'splitter': 'random', 'max_depth': 15

{'criterion': 'gini', 'splitter': 'random', 'max_depth': 27, 'min_samples_leaf': 2.9380319373672625} 0.8550859705282494


# Random Forest Classifier with different hyper parameter tuning techniques.

In [ ]:
# apply Randome Forest Classifier with GridSearch.

In [5]:
algo = ensemble.RandomForestClassifier()

model = model_selection.GridSearchCV(algo, param_grid = {
    'criterion': ['gini','entropy','log_loss'],
    'max_depth': [i for i in range(8,16)],
    "min_samples_leaf": [i for i in range(2,9)]
}, scoring = 'accuracy', cv = model_selection.StratifiedShuffleSplit(n_splits = 10))
model.fit(X,y)
print(model.best_params_)
print(model.best_score_)

{'criterion': 'gini', 'max_depth': 13, 'min_samples_leaf': 2}
0.9410179640718563


In [ ]:
# apply Randome Forest Classifier with RandomizedSearch.

In [7]:
algo = ensemble.RandomForestClassifier()
model = model_selection.RandomizedSearchCV(algo, param_distributions = {
    'criterion': ['gini','entropy','log_loss'],
    'max_depth': np.arange(8,33),
    "min_samples_leaf": np.arange(2,9)
},
scoring = 'accuracy', cv = model_selection.StratifiedShuffleSplit(n_splits = 10), n_iter = 15)

model.fit(X,y)
print(model.best_params_)
print(model.best_score_)

{'min_samples_leaf': 2, 'max_depth': 20, 'criterion': 'log_loss'}
0.9470059880239523


In [ ]:
# apply Random Forest Classifier with gp minimize.

In [41]:
def optimize(params, params_names, x, y):
    params = dict(zip(params_names, params))
    model = ensemble.RandomForestClassifier(**params)
    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return -1.0 * np.mean(accuracies)

param_spaces = [
    space.Categorical(['gini','entropy','log_loss'], name = 'criterion'),
    space.Integer(100,250,name = 'n_estimators'),
    space.Integer(8,33, name = 'max_depth'),
    space.Integer(2,9, name = 'min_samples_leaf')
]

param_names = ['criterion','n_estimators','max_depth','min_samples_leaf']

optimaization_function = partial(optimize, params_names = param_names, x = X, y = y)

result = gp_minimize(optimaization_function,dimensions = param_spaces, n_calls = 15, n_random_starts = 10, verbose = 10)
print(dict(zip(param_names, result.x)))

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 3.7001
Function value obtained: -0.9331
Current minimum: -0.9331
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 2.1776
Function value obtained: -0.9391
Current minimum: -0.9391
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 2.3309
Function value obtained: -0.9265
Current minimum: -0.9391
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 4.7147
Function value obtained: -0.9442
Current minimum: -0.9442
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 2.7793
Function value obtained: -0.9331
Current minimum: -0.9442
Iteration No: 6 started. 

In [ ]:
# apply Random Forest Classifier with hyperopt.

In [40]:
def optimize(params, x, y):
    model = ensemble.RandomForestClassifier(**params)
    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return -1.0 * np.mean(accuracies)

param_spaces = {
    'criterion': hp.choice('criterion',['gini','entropy','log_loss']),
    'n_estimators': scope.int(hp.quniform('n_estimators',100,250,1)),
    'max_depth': scope.int(hp.quniform('max_depth',8,33,1)),
    'min_samples_leaf': scope.int(hp.quniform('min_samples_leaf',2,9,1))
}

optimaization_function = partial(optimize,x = X, y = y)

result = fmin(fn = optimaization_function,space = param_spaces, max_evals = 15, trials = Trials(), algo = tpe.suggest)
print(result)

100%|██████████| 15/15 [00:52<00:00,  3.49s/trial, best loss: -0.9450914182548364]
{'criterion': 2, 'max_depth': 32.0, 'min_samples_leaf': 3.0, 'n_estimators': 197.0}


In [ ]:
# apply Random Forest Classifier with optuna.

In [10]:
def optimize(trial, x, y):
    criterion = trial.suggest_categorical('criterion', ['gini','entropy','log_loss'])
    n_estimators = trial.suggest_int('n_estimators', 100,250)
    max_depth = trial.suggest_int('max_depth', 8, 32)
    min_samples_leaf = trial.suggest_float('min_samples_leaf',2,8)
    model = ensemble.RandomForestClassifier(
        criterion = criterion,
        n_estimators = n_estimators,
        max_depth = max_depth,
        min_samples_leaf = min_samples_leaf
    )

    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return np.mean(accuracies)

optimaization_function = partial(optimize,x = X, y = y)

study = optuna.create_study(direction = 'maximize')
study.optimize(optimaization_function, n_trials = 15)
print(study.best_params, study.best_value)

[I 2022-08-12 09:49:02,018] A new study created in memory with name: no-name-616f5fc0-697e-4d04-b692-cafbf210a5d4
[I 2022-08-12 09:49:02,738] Trial 0 finished with value: 0.8550859705282494 and parameters: {'criterion': 'entropy', 'n_estimators': 121, 'max_depth': 16, 'min_samples_leaf': 3.1411082763075386}. Best is trial 0 with value: 0.8550859705282494.
[I 2022-08-12 09:49:03,577] Trial 1 finished with value: 0.8550859705282494 and parameters: {'criterion': 'entropy', 'n_estimators': 143, 'max_depth': 13, 'min_samples_leaf': 5.325889408850165}. Best is trial 0 with value: 0.8550859705282494.
[I 2022-08-12 09:49:04,619] Trial 2 finished with value: 0.8550859705282494 and parameters: {'criterion': 'gini', 'n_estimators': 183, 'max_depth': 32, 'min_samples_leaf': 5.2217551659083385}. Best is trial 0 with value: 0.8550859705282494.
[I 2022-08-12 09:49:05,718] Trial 3 finished with value: 0.8550859705282494 and parameters: {'criterion': 'entropy', 'n_estimators': 171, 'max_depth': 26, 'mi

{'criterion': 'entropy', 'n_estimators': 121, 'max_depth': 16, 'min_samples_leaf': 3.1411082763075386} 0.8550859705282494


# AdaBoost Classifier with different hyper parameter tuning techniques.

In [ ]:
# apply AdaBoost Classifier with GridSearch.

In [8]:
algo = ensemble.AdaBoostClassifier()
list = []
start, step, end = 0.01, 0.01, 0.11
while start < end:
    list.append(start)
    start += step
model = model_selection.GridSearchCV(algo, param_grid = {
    'learning_rate': list,
    'n_estimators': [i for i in range(50,100)],
    'algorithm': ['SAMME','SAMME.R']
},
scoring = 'accuracy', cv = model_selection.StratifiedShuffleSplit(n_splits = 10))

model.fit(X,y)
print(model.best_params_)
print(model.best_score_)

{'algorithm': 'SAMME.R', 'learning_rate': 0.10999999999999999, 'n_estimators': 91}
0.8721556886227546


In [ ]:
# apply AdaBoost Classifier with RandomizedSearch.

In [14]:
algo = ensemble.AdaBoostClassifier()
model = model_selection.RandomizedSearchCV(algo, param_distributions = {
    'learning_rate': np.arange(0.01,0.11,0.01),
    'n_estimators': np.arange(50,101),
    'algorithm': ['SAMME','SAMME.R']
},
scoring = 'accuracy', cv = model_selection.StratifiedShuffleSplit(n_splits = 10), n_iter = 15)

model.fit(X,y)
print(model.best_params_)
print(model.best_score_)

{'n_estimators': 90, 'learning_rate': 0.09999999999999999, 'algorithm': 'SAMME.R'}
0.8640718562874252


In [ ]:
# apply AdaBoost Classifier with gp minimize.

In [39]:
def optimize(params, params_names, x, y):
    params = dict(zip(params_names, params))
    model = ensemble.AdaBoostClassifier(**params)
    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return -1.0 * np.mean(accuracies)

param_spaces = [
    space.Real(0.01,0.11, name = 'learning_rate'),
    space.Integer(50,101, name = 'n_estimators'),
    space.Categorical(['SAMME','SAMME.R'], name = 'algorithm'),
]

param_names = ['learning_rate','n_estimators','algorithm']

optimaization_function = partial(optimize, params_names = param_names, x = X, y = y)

result = gp_minimize(optimaization_function,dimensions = param_spaces, n_calls = 15, n_random_starts = 10, verbose = 10)
print(dict(zip(param_names, result.x)))

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.9749
Function value obtained: -0.8590
Current minimum: -0.8590
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 1.2923
Function value obtained: -0.8566
Current minimum: -0.8590
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 1.1954
Function value obtained: -0.8587
Current minimum: -0.8590
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 1.3000
Function value obtained: -0.8599
Current minimum: -0.8599
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 1.0622
Function value obtained: -0.8590
Current minimum: -0.8599
Iteration No: 6 started. 

In [ ]:
# apply AdaBoost Classifier with hyperopt.

In [38]:
def optimize(params, x, y):
    model = ensemble.AdaBoostClassifier(**params)
    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return -1.0 * np.mean(accuracies)

param_spaces = {
    'n_estimators': scope.int(hp.quniform('n_estimators',50,101,1)),
    'learning_rate': hp.loguniform('learning_rate',np.log(0.01),np.log(0.11)),
    'algorithm': hp.choice('algorithm',['SAMME','SAMME.R'])
}

optimaization_function = partial(optimize,x = X, y = y)

result = fmin(fn = optimaization_function,space = param_spaces, max_evals = 15, trials = Trials(), algo = tpe.suggest)
print(result)

100%|██████████| 15/15 [00:15<00:00,  1.02s/trial, best loss: -0.864988226607417]
{'algorithm': 1, 'learning_rate': 0.09573031013833175, 'n_estimators': 100.0}


In [ ]:
# apply AdaBoost Classifier with optuna.

In [22]:
def optimize(trial, x, y):
    learning_rate = trial.suggest_float('learning_rate',0.01,0.1)
    n_estimators = trial.suggest_int('n_estimators', 50, 150)
    algorithm = trial.suggest_categorical('algorithm', ['SAMME','SAMME.R'])
    model = ensemble.AdaBoostClassifier(
        learning_rate = learning_rate,
        n_estimators = n_estimators,
        algorithm = algorithm
    )

    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return np.mean(accuracies)

optimaization_function = partial(optimize,x = X, y = y)

study = optuna.create_study(direction = 'maximize')
study.optimize(optimaization_function, n_trials = 15)
print(study.best_params, study.best_value)

[I 2022-08-12 09:57:30,106] A new study created in memory with name: no-name-7987f4db-1312-4c49-bff4-2bc6b7853401
[I 2022-08-12 09:57:31,588] Trial 0 finished with value: 0.8601883742813279 and parameters: {'learning_rate': 0.04492346222183832, 'n_estimators': 89, 'algorithm': 'SAMME'}. Best is trial 0 with value: 0.8601883742813279.
[I 2022-08-12 09:57:33,707] Trial 1 finished with value: 0.8577877727802765 and parameters: {'learning_rate': 0.08260026929562704, 'n_estimators': 137, 'algorithm': 'SAMME'}. Best is trial 0 with value: 0.8601883742813279.
[I 2022-08-12 09:57:35,321] Trial 2 finished with value: 0.8577877727802765 and parameters: {'learning_rate': 0.09578476201342846, 'n_estimators': 113, 'algorithm': 'SAMME'}. Best is trial 0 with value: 0.8601883742813279.
[I 2022-08-12 09:57:37,169] Trial 3 finished with value: 0.8574874724799763 and parameters: {'learning_rate': 0.09501675540925733, 'n_estimators': 130, 'algorithm': 'SAMME'}. Best is trial 0 with value: 0.8601883742813

{'learning_rate': 0.09938766623872351, 'n_estimators': 125, 'algorithm': 'SAMME.R'} 0.868889879384632


# Gradient Boosting Classifier with different hyper parameter tuning techniques.

In [ ]:
# apply Gradient Boosting Classifier with GridSearch.

In [11]:
algo = ensemble.GradientBoostingClassifier()
list = []
start, step, end = 0.1, 0.01, 0.21
while start < end:
    list.append(start)
    start += step
model = model_selection.GridSearchCV(algo, param_grid = {
    'loss': ['log_loss', 'deviance', 'exponential'],
    'learning_rate': list,
    'criterion': ['friedman_mse', 'squared_error', 'mse']
},
scoring = 'accuracy', cv = model_selection.StratifiedShuffleSplit(n_splits = 10))

model.fit(X,y)
print(model.best_params_)
print(model.best_score_)

{'criterion': 'squared_error', 'learning_rate': 0.11, 'loss': 'log_loss'}
0.9550898203592816


In [ ]:
# apply Gradient Boosting Classifier with RandomizedSearch.

In [10]:
algo = ensemble.GradientBoostingClassifier()
model = model_selection.RandomizedSearchCV(algo, param_distributions = {
    'loss': ['log_loss', 'deviance', 'exponential'],
    'learning_rate': np.arange(0.1,0.21,0.01),
    'criterion': ['friedman_mse', 'squared_error', 'mse']
},
scoring = 'accuracy', cv = model_selection.StratifiedShuffleSplit(n_splits = 10), n_iter = 15)

model.fit(X,y)
print(model.best_params_)
print(model.best_score_)

{'loss': 'exponential', 'learning_rate': 0.16999999999999998, 'criterion': 'mse'}
0.9535928143712576


In [ ]:
# apply Gradient Boosting Classifier with gp minimize.

In [43]:
def optimize(params, params_names, x, y):
    params = dict(zip(params_names, params))
    model = ensemble.GradientBoostingClassifier(**params)
    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return -1.0 * np.mean(accuracies)

param_spaces = [
    space.Categorical(['log_loss', 'deviance', 'exponential'], name = 'loss'),
    space.Integer(100,500, name = 'n_estimators'),
    space.Real(0.1,0.21,name = 'learning_rate'),
    space.Categorical(['friedman_mse', 'squared_error', 'mse'], name = 'criterion'),
    space.Integer(2,9, name = 'min_samples_leaf'),
    space.Integer(3,11, name = 'max_depth')
]

param_names = ['loss','n_estimators','learning_rate','criterion','min_samples_leaf','max_depth']

optimaization_function = partial(optimize, params_names = param_names, x = X, y = y)

result = gp_minimize(optimaization_function,dimensions = param_spaces, n_calls = 15, n_random_starts = 10, verbose = 10)
print(dict(zip(param_names, result.x)))

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 23.8518
Function value obtained: -0.9556
Current minimum: -0.9556
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 20.7378
Function value obtained: -0.9550
Current minimum: -0.9556
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 13.4217
Function value obtained: -0.9514
Current minimum: -0.9556
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 24.9866
Function value obtained: -0.9562
Current minimum: -0.9562
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 20.1820
Function value obtained: -0.9568
Current minimum: -0.9568
Iteration No: 6 star

In [ ]:
# apply Gradient Boosting Classifier with hyperopt.

In [36]:
def optimize(params, x, y):
    model = ensemble.GradientBoostingClassifier(**params)
    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return -1.0 * np.mean(accuracies)

param_spaces = {
    'loss': hp.choice('loss',['log_loss', 'deviance', 'exponential']),
    'n_estimators': scope.int(hp.quniform('n_estimators',100,500,1)),
    'learning_rate': hp.loguniform('learning_rate',np.log(0.1),np.log(0.2)),
    'criterion': hp.choice('criterion',['friedman_mse', 'squared_error', 'mse']),
    'min_samples_leaf': scope.int(hp.quniform('min_samples_leaf',2,9,1)),
    'max_depth': scope.int(hp.quniform('max_depth',3,11,1))
}

optimaization_function = partial(optimize,x = X, y = y)

result = fmin(fn = optimaization_function,space = param_spaces, max_evals = 15, trials = Trials(), algo = tpe.suggest)
print(result)

100%|██████████| 15/15 [04:04<00:00, 16.28s/trial, best loss: -0.9573960767364067]
{'criterion': 0, 'learning_rate': 0.1286777986645684, 'loss': 1, 'max_depth': 10.0, 'min_samples_leaf': 8.0, 'n_estimators': 443.0}


In [ ]:
# apply Gradient Boosting Classifier with optuna.

In [29]:
def optimize(trial, x, y):
    loss = trial.suggest_categorical('loss',['log_loss', 'deviance', 'exponential'])
    n_estimators =  trial.suggest_int('n_estimators', 100, 500)
    learning_rate = trial.suggest_float('learning_rate',0.1,0.2)
    criterion = trial.suggest_categorical('criterion',['friedman_mse', 'squared_error', 'mse'])
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 8)
    max_depth = trial.suggest_int('max_depth', 8, 32)
    model = ensemble.GradientBoostingClassifier(
        loss = loss,
        n_estimators = n_estimators,
        learning_rate = learning_rate,
        criterion = criterion,
        min_samples_leaf = min_samples_leaf,
        max_depth = max_depth
    )

    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return np.mean(accuracies)

optimaization_function = partial(optimize,x = X, y = y)

study = optuna.create_study(direction = 'maximize')
study.optimize(optimaization_function, n_trials = 15)
print(study.best_params, study.best_value)

[I 2022-08-12 11:14:02,890] A new study created in memory with name: no-name-21500ded-9a9a-4018-bad4-6a318e503b3e
[I 2022-08-12 11:14:12,889] Trial 0 finished with value: 0.9561966764365565 and parameters: {'loss': 'log_loss', 'n_estimators': 104, 'learning_rate': 0.12624703072360924, 'criterion': 'squared_error', 'min_samples_leaf': 7, 'max_depth': 12}. Best is trial 0 with value: 0.9561966764365565.
[I 2022-08-12 11:14:43,880] Trial 1 finished with value: 0.9552939746343044 and parameters: {'loss': 'log_loss', 'n_estimators': 215, 'learning_rate': 0.1471742148452869, 'criterion': 'friedman_mse', 'min_samples_leaf': 7, 'max_depth': 24}. Best is trial 0 with value: 0.9561966764365565.
[I 2022-08-12 11:15:07,845] Trial 2 finished with value: 0.9507971239605423 and parameters: {'loss': 'log_loss', 'n_estimators': 142, 'learning_rate': 0.1293398398717019, 'criterion': 'mse', 'min_samples_leaf': 2, 'max_depth': 19}. Best is trial 0 with value: 0.9561966764365565.
[I 2022-08-12 11:15:36,033

{'loss': 'deviance', 'n_estimators': 430, 'learning_rate': 0.12377851019936359, 'criterion': 'squared_error', 'min_samples_leaf': 3, 'max_depth': 9} 0.9579957768863314


# XGB Classifier with different hyper parameter tuning techniques.

In [ ]:
# apply XGB Classifier with GridSearch.

In [7]:
algo = xgb.XGBClassifier()
list = []
start, step, end = 0.1, 0.01, 0.2
while start < end:
    list.append(start)
    start += step
model = model_selection.GridSearchCV(algo, param_grid = {
    'sampling_method': ['uniform','gradient_based'],
    'eta': list
},
scoring = 'accuracy', cv = model_selection.StratifiedShuffleSplit(n_splits = 10))

model.fit(X,y)
print(model.best_params_)
print(model.best_score_)

{'eta': 0.14, 'sampling_method': 'uniform'}
0.9556886227544912


In [ ]:
# apply XGB Classifier with RandomizedSearch.

In [9]:
algo = xgb.XGBClassifier()
model = model_selection.RandomizedSearchCV(algo, param_distributions = {
    'sampling_method': ['uniform','gradient_based'],
    'eta': np.arange(0.1,0.2,0.01)
},
scoring = 'accuracy', cv = model_selection.StratifiedShuffleSplit(n_splits = 10), n_iter = 15)

model.fit(X,y)
print(model.best_params_)
print(model.best_score_)

{'sampling_method': 'uniform', 'eta': 0.13999999999999999}
0.9595808383233534


In [ ]:
# apply XGB Classifier with gp minimize.

In [30]:
def optimize(params, params_names, x, y):
    params = dict(zip(params_names, params))
    model = xgb.XGBClassifier(**params)
    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return -1.0 * np.mean(accuracies)

param_spaces = [
    space.Categorical(['uniform','gradient_based'], name = 'sampling_method'),
    space.Real(0.1,0.21,name = 'eta'),
    space.Integer(3,11, name = 'max_depth')
]

param_names = ['sampling_method','eta','max_depth']

optimaization_function = partial(optimize, params_names = param_names, x = X, y = y)

result = gp_minimize(optimaization_function,dimensions = param_spaces, n_calls = 15, n_random_starts = 10, verbose = 10)
print(dict(zip(param_names, result.x)))

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 2.0869
Function value obtained: -0.9571
Current minimum: -0.9571
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 1.9003
Function value obtained: -0.9526
Current minimum: -0.9571
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 1.6405
Function value obtained: -0.9508
Current minimum: -0.9571
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 2.1810
Function value obtained: -0.9544
Current minimum: -0.9571
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 2.1920
Function value obtained: -0.9538
Current minimum: -0.9571
Iteration No: 6 started. 

In [ ]:
# apply XGB Classifier with hyperopt.

In [33]:
def optimize(params, x, y):
    model = xgb.XGBClassifier(**params)
    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return -1.0 * np.mean(accuracies)

param_spaces = {
    'sampling_method': hp.choice('sampling_method',['uniform','gradient_based']),
    'eta': hp.loguniform('eta',np.log(0.1),np.log(1)),
    'max_depth': scope.int(hp.quniform('max_depth',3,11,1))
}

optimaization_function = partial(optimize,x = X, y = y)

result = fmin(fn = optimaization_function,space = param_spaces, max_evals = 15, trials = Trials(), algo = tpe.suggest)
print(result)

100%|██████████| 15/15 [00:30<00:00,  2.04s/trial, best loss: -0.9561953257605431]
{'eta': 0.10374145404246778, 'max_depth': 4.0, 'sampling_method': 0}


In [ ]:
# apply XGB Classifier with optuna.

In [32]:
def optimize(trial, x, y):
    sampling_method = trial.suggest_categorical('sampling_method',['uniform','gradient_based'])
    eta = trial.suggest_float('eta',0.1,0.9)
    max_depth = trial.suggest_int('max_depth', 8, 32)
    model = xgb.XGBClassifier(
        sampling_method = sampling_method,
        eta = eta,
        max_depth = max_depth,
    )

    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return np.mean(accuracies)

optimaization_function = partial(optimize,x = X, y = y)

study = optuna.create_study(direction = 'maximize')
study.optimize(optimaization_function, n_trials = 15)
print(study.best_params, study.best_value)

[I 2022-08-12 11:23:26,463] A new study created in memory with name: no-name-e3eb104c-c327-404a-9b47-e7fa4ea098ca
[I 2022-08-12 11:23:28,672] Trial 0 finished with value: 0.9510974242608425 and parameters: {'sampling_method': 'uniform', 'eta': 0.6812719497438209, 'max_depth': 15}. Best is trial 0 with value: 0.9510974242608425.
[I 2022-08-12 11:23:30,916] Trial 1 finished with value: 0.9525948737343042 and parameters: {'sampling_method': 'gradient_based', 'eta': 0.6542760340605527, 'max_depth': 19}. Best is trial 1 with value: 0.9525948737343042.
[I 2022-08-12 11:23:34,041] Trial 2 finished with value: 0.9501965233599418 and parameters: {'sampling_method': 'uniform', 'eta': 0.7309114150204372, 'max_depth': 18}. Best is trial 1 with value: 0.9525948737343042.
[I 2022-08-12 11:23:37,892] Trial 3 finished with value: 0.9525944235089663 and parameters: {'sampling_method': 'uniform', 'eta': 0.29166320468540385, 'max_depth': 22}. Best is trial 1 with value: 0.9525948737343042.
[I 2022-08-12 

{'sampling_method': 'gradient_based', 'eta': 0.3310868528024605, 'max_depth': 25} 0.9546951749350552


# Cat Boost Classifier with different hyper parameter tuning techniques.

In [ ]:
# apply Cat Boost Classifier with gp minimize.

In [44]:
def optimize(params, params_names, x, y):
    params = dict(zip(params_names, params))
    model = catboost.CatBoostClassifier(**params)
    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return -1.0 * np.mean(accuracies)

param_spaces = [
    space.Real(0.01,0.11,name = 'learning_rate'),
    space.Integer(4,16, name = 'depth'),
    space.Integer(50,151, name = 'iterations'),
    space.Integer(1,5, name = 'l2_leaf_reg')
]

param_names = ['learning_rate','depth','iterations','l2_leaf_reg']

optimaization_function = partial(optimize, params_names = param_names, x = X, y = y)

result = gp_minimize(optimaization_function,dimensions = param_spaces, n_calls = 15, n_random_starts = 10, verbose = 10)
print(dict(zip(param_names, result.x)))

Iteration No: 1 started. Evaluating function at random point.
0:	learn: 0.6787996	total: 69.3ms	remaining: 3.95s
1:	learn: 0.6705439	total: 106ms	remaining: 2.98s
2:	learn: 0.6581184	total: 127ms	remaining: 2.34s
3:	learn: 0.6472485	total: 147ms	remaining: 1.99s
4:	learn: 0.6380866	total: 167ms	remaining: 1.77s
5:	learn: 0.6281431	total: 187ms	remaining: 1.62s
6:	learn: 0.6176023	total: 207ms	remaining: 1.51s
7:	learn: 0.6069789	total: 208ms	remaining: 1.3s
8:	learn: 0.5979651	total: 228ms	remaining: 1.24s
9:	learn: 0.5891653	total: 247ms	remaining: 1.19s
10:	learn: 0.5797399	total: 268ms	remaining: 1.14s
11:	learn: 0.5734509	total: 289ms	remaining: 1.11s
12:	learn: 0.5653678	total: 309ms	remaining: 1.07s
13:	learn: 0.5577432	total: 333ms	remaining: 1.05s
14:	learn: 0.5502185	total: 355ms	remaining: 1.02s
15:	learn: 0.5430394	total: 375ms	remaining: 985ms
16:	learn: 0.5376277	total: 395ms	remaining: 953ms
17:	learn: 0.5305272	total: 415ms	remaining: 921ms
18:	learn: 0.5233165	total: 43

49:	learn: 0.3747951	total: 873ms	remaining: 140ms
50:	learn: 0.3715944	total: 895ms	remaining: 123ms
51:	learn: 0.3687832	total: 916ms	remaining: 106ms
52:	learn: 0.3651831	total: 934ms	remaining: 88.1ms
53:	learn: 0.3622083	total: 953ms	remaining: 70.6ms
54:	learn: 0.3593693	total: 973ms	remaining: 53.1ms
55:	learn: 0.3565575	total: 993ms	remaining: 35.5ms
56:	learn: 0.3536944	total: 1.01s	remaining: 17.8ms
57:	learn: 0.3515211	total: 1.04s	remaining: 0us
0:	learn: 0.6790053	total: 3.67ms	remaining: 209ms
1:	learn: 0.6707831	total: 29.9ms	remaining: 838ms
2:	learn: 0.6583702	total: 55.1ms	remaining: 1.01s
3:	learn: 0.6458701	total: 60.6ms	remaining: 818ms
4:	learn: 0.6346532	total: 81.6ms	remaining: 865ms
5:	learn: 0.6233505	total: 101ms	remaining: 878ms
6:	learn: 0.6145916	total: 121ms	remaining: 878ms
7:	learn: 0.6052333	total: 140ms	remaining: 872ms
8:	learn: 0.5964188	total: 140ms	remaining: 763ms
9:	learn: 0.5886758	total: 159ms	remaining: 764ms
10:	learn: 0.5806732	total: 178ms

45:	learn: 0.1770554	total: 557ms	remaining: 230ms
46:	learn: 0.1747355	total: 569ms	remaining: 218ms
47:	learn: 0.1723347	total: 580ms	remaining: 206ms
48:	learn: 0.1706095	total: 592ms	remaining: 193ms
49:	learn: 0.1685349	total: 606ms	remaining: 182ms
50:	learn: 0.1671325	total: 622ms	remaining: 171ms
51:	learn: 0.1657040	total: 636ms	remaining: 159ms
52:	learn: 0.1641071	total: 643ms	remaining: 146ms
53:	learn: 0.1626712	total: 658ms	remaining: 134ms
54:	learn: 0.1608213	total: 674ms	remaining: 122ms
55:	learn: 0.1592233	total: 690ms	remaining: 111ms
56:	learn: 0.1574513	total: 704ms	remaining: 98.8ms
57:	learn: 0.1559263	total: 717ms	remaining: 86.5ms
58:	learn: 0.1545441	total: 727ms	remaining: 74ms
59:	learn: 0.1531855	total: 737ms	remaining: 61.5ms
60:	learn: 0.1525230	total: 738ms	remaining: 48.4ms
61:	learn: 0.1511833	total: 749ms	remaining: 36.2ms
62:	learn: 0.1499110	total: 760ms	remaining: 24.1ms
63:	learn: 0.1485308	total: 771ms	remaining: 12ms
64:	learn: 0.1473243	total:

13:	learn: 0.3298064	total: 227ms	remaining: 826ms
14:	learn: 0.3195803	total: 237ms	remaining: 790ms
15:	learn: 0.3089879	total: 247ms	remaining: 756ms
16:	learn: 0.2988280	total: 259ms	remaining: 732ms
17:	learn: 0.2904741	total: 272ms	remaining: 710ms
18:	learn: 0.2820578	total: 283ms	remaining: 684ms
19:	learn: 0.2754110	total: 293ms	remaining: 659ms
20:	learn: 0.2675052	total: 302ms	remaining: 634ms
21:	learn: 0.2602617	total: 313ms	remaining: 611ms
22:	learn: 0.2540541	total: 323ms	remaining: 589ms
23:	learn: 0.2484370	total: 333ms	remaining: 568ms
24:	learn: 0.2429586	total: 342ms	remaining: 547ms
25:	learn: 0.2404460	total: 352ms	remaining: 528ms
26:	learn: 0.2345145	total: 362ms	remaining: 509ms
27:	learn: 0.2303027	total: 372ms	remaining: 491ms
28:	learn: 0.2262519	total: 381ms	remaining: 474ms
29:	learn: 0.2230075	total: 391ms	remaining: 456ms
30:	learn: 0.2204317	total: 392ms	remaining: 430ms
31:	learn: 0.2168908	total: 402ms	remaining: 414ms
32:	learn: 0.2136460	total: 411

46:	learn: 0.3869114	total: 759ms	remaining: 662ms
47:	learn: 0.3826957	total: 779ms	remaining: 649ms
48:	learn: 0.3789968	total: 801ms	remaining: 638ms
49:	learn: 0.3751097	total: 821ms	remaining: 624ms
50:	learn: 0.3716146	total: 827ms	remaining: 600ms
51:	learn: 0.3686219	total: 846ms	remaining: 586ms
52:	learn: 0.3653809	total: 865ms	remaining: 572ms
53:	learn: 0.3620683	total: 885ms	remaining: 557ms
54:	learn: 0.3594955	total: 904ms	remaining: 543ms
55:	learn: 0.3562948	total: 923ms	remaining: 528ms
56:	learn: 0.3532946	total: 942ms	remaining: 512ms
57:	learn: 0.3500681	total: 961ms	remaining: 497ms
58:	learn: 0.3470358	total: 982ms	remaining: 483ms
59:	learn: 0.3439463	total: 1s	remaining: 468ms
60:	learn: 0.3410088	total: 1.02s	remaining: 453ms
61:	learn: 0.3381872	total: 1.04s	remaining: 438ms
62:	learn: 0.3351909	total: 1.06s	remaining: 421ms
63:	learn: 0.3323781	total: 1.08s	remaining: 405ms
64:	learn: 0.3297217	total: 1.1s	remaining: 390ms
65:	learn: 0.3267704	total: 1.12s	r

...

83:	learn: 0.4609194	total: 30.6s	remaining: 24.9s
84:	learn: 0.4594453	total: 30.8s	remaining: 24.3s
85:	learn: 0.4571468	total: 30.8s	remaining: 23.6s
86:	learn: 0.4559509	total: 31.5s	remaining: 23.5s
87:	learn: 0.4550533	total: 32.1s	remaining: 23.3s
88:	learn: 0.4540483	total: 32.9s	remaining: 23.2s
89:	learn: 0.4533078	total: 33.5s	remaining: 23s
90:	learn: 0.4523233	total: 34.3s	remaining: 22.9s
91:	learn: 0.4502383	total: 34.4s	remaining: 22.2s
92:	learn: 0.4494164	total: 35s	remaining: 22.1s
93:	learn: 0.4474522	total: 35.4s	remaining: 21.6s
94:	learn: 0.4454322	total: 35.4s	remaining: 21s
95:	learn: 0.4436671	total: 36.1s	remaining: 20.8s
96:	learn: 0.4415639	total: 36.1s	remaining: 20.2s
97:	learn: 0.4406718	total: 36.8s	remaining: 19.9s
98:	learn: 0.4398376	total: 37.5s	remaining: 19.7s
99:	learn: 0.4392437	total: 38.2s	remaining: 19.5s
100:	learn: 0.4380444	total: 38.9s	remaining: 19.3s
101:	learn: 0.4369994	total: 39.6s	remaining: 19s
102:	learn: 0.4361002	total: 40.3s	re

96:	learn: 0.4460241	total: 39.1s	remaining: 21.9s
97:	learn: 0.4436467	total: 39.1s	remaining: 21.2s
98:	learn: 0.4428696	total: 39.8s	remaining: 20.9s
99:	learn: 0.4407633	total: 39.8s	remaining: 20.3s
100:	learn: 0.4398569	total: 40.5s	remaining: 20.1s
101:	learn: 0.4389133	total: 41.2s	remaining: 19.8s
102:	learn: 0.4367092	total: 41.3s	remaining: 19.2s
103:	learn: 0.4359422	total: 41.9s	remaining: 18.9s
104:	learn: 0.4334801	total: 42s	remaining: 18.3s
105:	learn: 0.4323034	total: 42.6s	remaining: 18s
106:	learn: 0.4304704	total: 42.7s	remaining: 17.4s
107:	learn: 0.4296417	total: 43.4s	remaining: 17.1s
108:	learn: 0.4273417	total: 43.4s	remaining: 16.5s
109:	learn: 0.4253311	total: 43.4s	remaining: 16s
110:	learn: 0.4236442	total: 43.5s	remaining: 15.4s
111:	learn: 0.4228575	total: 44.2s	remaining: 15.1s
112:	learn: 0.4206750	total: 44.2s	remaining: 14.6s
113:	learn: 0.4192208	total: 44.9s	remaining: 14.3s
114:	learn: 0.4175069	total: 44.9s	remaining: 13.8s
115:	learn: 0.4157003	

0:	learn: 0.6872480	total: 2.45ms	remaining: 336ms
1:	learn: 0.6817923	total: 5.55ms	remaining: 377ms
2:	learn: 0.6760619	total: 8.05ms	remaining: 362ms
3:	learn: 0.6711134	total: 10.1ms	remaining: 340ms
4:	learn: 0.6655188	total: 12.6ms	remaining: 336ms
5:	learn: 0.6598047	total: 14.9ms	remaining: 327ms
6:	learn: 0.6544242	total: 17.4ms	remaining: 325ms
7:	learn: 0.6493411	total: 19.1ms	remaining: 311ms
8:	learn: 0.6438393	total: 21.6ms	remaining: 309ms
9:	learn: 0.6386281	total: 24ms	remaining: 307ms
10:	learn: 0.6333569	total: 26.1ms	remaining: 302ms
11:	learn: 0.6284511	total: 28.4ms	remaining: 298ms
12:	learn: 0.6235165	total: 30.5ms	remaining: 294ms
13:	learn: 0.6189934	total: 32.9ms	remaining: 292ms
14:	learn: 0.6141765	total: 35.8ms	remaining: 294ms
15:	learn: 0.6099841	total: 38.4ms	remaining: 293ms
16:	learn: 0.6056125	total: 40.7ms	remaining: 290ms
17:	learn: 0.6010199	total: 43.3ms	remaining: 289ms
18:	learn: 0.5964923	total: 45.7ms	remaining: 286ms
19:	learn: 0.5919537	tot

113:	learn: 0.3672942	total: 181ms	remaining: 38.1ms
114:	learn: 0.3657971	total: 182ms	remaining: 36.3ms
115:	learn: 0.3648243	total: 182ms	remaining: 34.6ms
116:	learn: 0.3636809	total: 184ms	remaining: 33.1ms
117:	learn: 0.3622372	total: 185ms	remaining: 31.4ms
118:	learn: 0.3610465	total: 186ms	remaining: 29.7ms
119:	learn: 0.3598526	total: 187ms	remaining: 28ms
120:	learn: 0.3586811	total: 188ms	remaining: 26.4ms
121:	learn: 0.3575287	total: 188ms	remaining: 24.7ms
122:	learn: 0.3564247	total: 189ms	remaining: 23.1ms
123:	learn: 0.3551361	total: 190ms	remaining: 21.4ms
124:	learn: 0.3540116	total: 190ms	remaining: 19.8ms
125:	learn: 0.3528162	total: 191ms	remaining: 18.2ms
126:	learn: 0.3516575	total: 192ms	remaining: 16.6ms
127:	learn: 0.3503645	total: 192ms	remaining: 15ms
128:	learn: 0.3492065	total: 193ms	remaining: 13.5ms
129:	learn: 0.3479636	total: 194ms	remaining: 11.9ms
130:	learn: 0.3467961	total: 194ms	remaining: 10.4ms
131:	learn: 0.3455277	total: 195ms	remaining: 8.86

0:	learn: 0.6871282	total: 2.8ms	remaining: 384ms
1:	learn: 0.6818254	total: 6.25ms	remaining: 425ms
2:	learn: 0.6761706	total: 8.68ms	remaining: 391ms
3:	learn: 0.6708125	total: 11.3ms	remaining: 379ms
4:	learn: 0.6653493	total: 13.9ms	remaining: 369ms
5:	learn: 0.6600448	total: 16.4ms	remaining: 360ms
6:	learn: 0.6547677	total: 18.6ms	remaining: 348ms
7:	learn: 0.6494658	total: 21.2ms	remaining: 345ms
8:	learn: 0.6448769	total: 23.6ms	remaining: 338ms
9:	learn: 0.6398558	total: 26.2ms	remaining: 335ms
10:	learn: 0.6349040	total: 28.3ms	remaining: 327ms
11:	learn: 0.6304066	total: 31ms	remaining: 325ms
12:	learn: 0.6251215	total: 33.8ms	remaining: 325ms
13:	learn: 0.6203366	total: 36.2ms	remaining: 321ms
14:	learn: 0.6154567	total: 38.9ms	remaining: 319ms
15:	learn: 0.6112016	total: 41.2ms	remaining: 314ms
16:	learn: 0.6065464	total: 43.5ms	remaining: 310ms
17:	learn: 0.6022137	total: 46ms	remaining: 307ms
18:	learn: 0.5974477	total: 48.3ms	remaining: 303ms
19:	learn: 0.5933017	total:

In [ ]:
# apply Cat Boost Classifier with hyperopt.

In [45]:
def optimize(params, x, y):
    model = catboost.CatBoostClassifier(**params)
    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return -1.0 * np.mean(accuracies)

param_spaces = {
    'learning_rate':hp.loguniform('learning_rate',np.log(0.01),np.log(0.11)),
    'depth': scope.int(hp.quniform('depth',3,11,1)),
    'iterations': scope.int(hp.quniform('iterations',50,151,1)),
    'l2_leaf_reg': scope.int(hp.quniform('l2_leaf_reg',1,5,1))
}

optimaization_function = partial(optimize,x = X, y = y)

result = fmin(fn = optimaization_function,space = param_spaces, max_evals = 15, trials = Trials(), algo = tpe.suggest)
print(result)

  0%|          | 0/15 [00:00<?, ?trial/s, best loss=?]0:	learn: 0.6773909	total: 8.47ms	remaining: 762ms
1:	learn: 0.6663771	total: 32.5ms	remaining: 1.45s
2:	learn: 0.6538106	total: 46.2ms	remaining: 1.35s
3:	learn: 0.6412181	total: 52.2ms	remaining: 1.14s
4:	learn: 0.6281807	total: 63.1ms	remaining: 1.09s
5:	learn: 0.6180733	total: 74.1ms	remaining: 1.05s
6:	learn: 0.6099949	total: 86.1ms	remaining: 1.03s
7:	learn: 0.6003721	total: 97.8ms	remaining: 1.01s
8:	learn: 0.5917868	total: 109ms	remaining: 990ms
9:	learn: 0.5812146	total: 120ms	remaining: 972ms
10:	learn: 0.5704612	total: 123ms	remaining: 893ms
11:	learn: 0.5620785	total: 133ms	remaining: 876ms
12:	learn: 0.5535967	total: 142ms	remaining: 855ms
13:	learn: 0.5445517	total: 147ms	remaining: 811ms
14:	learn: 0.5352301	total: 152ms	remaining: 773ms
15:	learn: 0.5287098	total: 162ms	remaining: 759ms
16:	learn: 0.5202242	total: 171ms	remaining: 745ms
17:	learn: 0.5135735	total: 181ms	remaining: 733ms
18:	learn: 0.5075813	total: 19

86:	learn: 0.2760504	total: 807ms	remaining: 37.1ms
87:	learn: 0.2746674	total: 819ms	remaining: 27.9ms
88:	learn: 0.2731273	total: 830ms	remaining: 18.6ms
89:	learn: 0.2716644	total: 841ms	remaining: 9.35ms
90:	learn: 0.2704043	total: 854ms	remaining: 0us
0:	learn: 0.6815259	total: 24.7ms	remaining: 2.22s
1:	learn: 0.6716897	total: 48.4ms	remaining: 2.15s
2:	learn: 0.6609204	total: 72.2ms	remaining: 2.12s
3:	learn: 0.6483002	total: 96ms	remaining: 2.09s
4:	learn: 0.6374882	total: 120ms	remaining: 2.07s
5:	learn: 0.6269539	total: 140ms	remaining: 1.99s
6:	learn: 0.6162722	total: 153ms	remaining: 1.83s
7:	learn: 0.6063710	total: 159ms	remaining: 1.65s
8:	learn: 0.5963453	total: 169ms	remaining: 1.54s
9:	learn: 0.5873149	total: 179ms	remaining: 1.45s
10:	learn: 0.5786376	total: 188ms	remaining: 1.37s
11:	learn: 0.5686827	total: 191ms	remaining: 1.26s
12:	learn: 0.5603321	total: 201ms	remaining: 1.21s
13:	learn: 0.5523138	total: 211ms	remaining: 1.16s
14:	learn: 0.5457052	total: 222ms	rem

76:	learn: 0.2963920	total: 813ms	remaining: 148ms
77:	learn: 0.2946219	total: 824ms	remaining: 137ms
78:	learn: 0.2932925	total: 834ms	remaining: 127ms
79:	learn: 0.2914801	total: 847ms	remaining: 116ms
80:	learn: 0.2893994	total: 858ms	remaining: 106ms
81:	learn: 0.2872541	total: 868ms	remaining: 95.3ms
82:	learn: 0.2856191	total: 878ms	remaining: 84.6ms
83:	learn: 0.2837683	total: 889ms	remaining: 74ms
84:	learn: 0.2820047	total: 899ms	remaining: 63.4ms
85:	learn: 0.2807850	total: 909ms	remaining: 52.8ms
86:	learn: 0.2786991	total: 910ms	remaining: 41.9ms
87:	learn: 0.2773628	total: 921ms	remaining: 31.4ms
88:	learn: 0.2757471	total: 931ms	remaining: 20.9ms
89:	learn: 0.2744153	total: 940ms	remaining: 10.4ms
90:	learn: 0.2731806	total: 950ms	remaining: 0us
0:	learn: 0.6776304	total: 8.15ms	remaining: 734ms
1:	learn: 0.6668303	total: 31.5ms	remaining: 1.4s
2:	learn: 0.6550786	total: 54.5ms	remaining: 1.6s
3:	learn: 0.6425398	total: 68.4ms	remaining: 1.49s
4:	learn: 0.6314849	total: 9

0:	learn: 0.6806543	total: 3.96ms	remaining: 530ms
1:	learn: 0.6701155	total: 7.88ms	remaining: 524ms
2:	learn: 0.6595080	total: 11.3ms	remaining: 497ms
3:	learn: 0.6484747	total: 14.6ms	remaining: 477ms
4:	learn: 0.6383385	total: 17.8ms	remaining: 462ms
5:	learn: 0.6272186	total: 21.1ms	remaining: 453ms
6:	learn: 0.6170353	total: 24.5ms	remaining: 447ms
7:	learn: 0.6078369	total: 28.2ms	remaining: 448ms
8:	learn: 0.5984764	total: 31.6ms	remaining: 442ms
9:	learn: 0.5895792	total: 35.2ms	remaining: 440ms
10:	learn: 0.5807076	total: 39ms	remaining: 440ms
11:	learn: 0.5721574	total: 42.6ms	remaining: 437ms
12:	learn: 0.5643816	total: 46.2ms	remaining: 433ms
13:	learn: 0.5563929	total: 49.7ms	remaining: 430ms
14:	learn: 0.5482452	total: 53.2ms	remaining: 426ms
15:	learn: 0.5398111	total: 56.5ms	remaining: 420ms
16:	learn: 0.5319481	total: 59.9ms	remaining: 416ms
17:	learn: 0.5254134	total: 63.1ms	remaining: 410ms
18:	learn: 0.5185579	total: 66.7ms	remaining: 407ms
19:	learn: 0.5121102	tot

95:	learn: 0.2772307	total: 187ms	remaining: 75.9ms
96:	learn: 0.2761732	total: 188ms	remaining: 73.5ms
97:	learn: 0.2749067	total: 189ms	remaining: 71.2ms
98:	learn: 0.2733381	total: 189ms	remaining: 68.9ms
99:	learn: 0.2721414	total: 190ms	remaining: 66.6ms
100:	learn: 0.2706416	total: 191ms	remaining: 64.4ms
101:	learn: 0.2693117	total: 192ms	remaining: 62.1ms
102:	learn: 0.2680540	total: 193ms	remaining: 59.9ms
103:	learn: 0.2666963	total: 194ms	remaining: 57.8ms
104:	learn: 0.2655874	total: 195ms	remaining: 55.6ms
105:	learn: 0.2645634	total: 195ms	remaining: 53.5ms
106:	learn: 0.2632185	total: 196ms	remaining: 51.4ms
107:	learn: 0.2624829	total: 197ms	remaining: 49.3ms
108:	learn: 0.2616347	total: 198ms	remaining: 47.2ms
109:	learn: 0.2607220	total: 199ms	remaining: 45.2ms
110:	learn: 0.2595368	total: 200ms	remaining: 43.2ms
111:	learn: 0.2582971	total: 201ms	remaining: 41.2ms
112:	learn: 0.2575439	total: 201ms	remaining: 39.2ms
113:	learn: 0.2565074	total: 202ms	remaining: 37.3m

...

 87%|████████▋ | 13/15 [01:04<00:08,  4.30s/trial, best loss: -0.9546960753857305]0:	learn: 0.6853530	total: 8.8ms	remaining: 686ms
1:	learn: 0.6794213	total: 19.2ms	remaining: 739ms
2:	learn: 0.6719933	total: 28.2ms	remaining: 714ms
3:	learn: 0.6654687	total: 37.1ms	remaining: 695ms
4:	learn: 0.6594286	total: 47.1ms	remaining: 697ms
5:	learn: 0.6525788	total: 55.8ms	remaining: 678ms
6:	learn: 0.6461461	total: 59.5ms	remaining: 612ms
7:	learn: 0.6397622	total: 68.4ms	remaining: 607ms
8:	learn: 0.6337236	total: 77.6ms	remaining: 603ms
9:	learn: 0.6274580	total: 89.2ms	remaining: 615ms
10:	learn: 0.6212793	total: 97.7ms	remaining: 604ms
11:	learn: 0.6155304	total: 104ms	remaining: 578ms
12:	learn: 0.6098881	total: 108ms	remaining: 550ms
13:	learn: 0.6045101	total: 113ms	remaining: 523ms
14:	learn: 0.5992049	total: 116ms	remaining: 493ms
15:	learn: 0.5939852	total: 120ms	remaining: 473ms
16:	learn: 0.5884123	total: 124ms	remaining: 453ms
17:	learn: 0.5833234	total: 127ms	remaining: 431ms


45:	learn: 0.4681183	total: 201ms	remaining: 144ms
46:	learn: 0.4648951	total: 202ms	remaining: 138ms
47:	learn: 0.4614773	total: 205ms	remaining: 133ms
48:	learn: 0.4586050	total: 208ms	remaining: 128ms
49:	learn: 0.4556065	total: 211ms	remaining: 122ms
50:	learn: 0.4528458	total: 214ms	remaining: 117ms
51:	learn: 0.4494748	total: 216ms	remaining: 112ms
52:	learn: 0.4463601	total: 219ms	remaining: 107ms
53:	learn: 0.4436477	total: 222ms	remaining: 103ms
54:	learn: 0.4403875	total: 224ms	remaining: 97.9ms
55:	learn: 0.4380949	total: 225ms	remaining: 92.4ms
56:	learn: 0.4353989	total: 228ms	remaining: 87.9ms
57:	learn: 0.4325601	total: 232ms	remaining: 83.9ms
58:	learn: 0.4294170	total: 235ms	remaining: 79.6ms
59:	learn: 0.4266200	total: 237ms	remaining: 75.2ms
60:	learn: 0.4235935	total: 241ms	remaining: 71.3ms
61:	learn: 0.4209644	total: 244ms	remaining: 67ms
62:	learn: 0.4184341	total: 247ms	remaining: 62.8ms
63:	learn: 0.4158008	total: 250ms	remaining: 58.6ms
64:	learn: 0.4133316	to

72:	learn: 0.3950005	total: 197ms	remaining: 16.2ms
73:	learn: 0.3924069	total: 201ms	remaining: 13.6ms
74:	learn: 0.3899105	total: 204ms	remaining: 10.9ms
75:	learn: 0.3876398	total: 206ms	remaining: 8.13ms
76:	learn: 0.3850494	total: 209ms	remaining: 5.43ms
77:	learn: 0.3827253	total: 210ms	remaining: 2.69ms
78:	learn: 0.3805754	total: 213ms	remaining: 0us
 93%|█████████▎| 14/15 [01:12<00:05,  5.56s/trial, best loss: -0.9546960753857305]0:	learn: 0.6804302	total: 7.52ms	remaining: 677ms
1:	learn: 0.6707824	total: 11.2ms	remaining: 499ms
2:	learn: 0.6588281	total: 14.9ms	remaining: 436ms
3:	learn: 0.6485903	total: 18.4ms	remaining: 401ms
4:	learn: 0.6390390	total: 22.2ms	remaining: 382ms
5:	learn: 0.6286097	total: 25.8ms	remaining: 365ms
6:	learn: 0.6186173	total: 27.2ms	remaining: 326ms
7:	learn: 0.6097045	total: 30.7ms	remaining: 319ms
8:	learn: 0.6011491	total: 34.3ms	remaining: 313ms
9:	learn: 0.5917955	total: 37.9ms	remaining: 307ms
10:	learn: 0.5830032	total: 41.7ms	remaining: 3

0:	learn: 0.6812836	total: 4.87ms	remaining: 439ms
1:	learn: 0.6710805	total: 8.86ms	remaining: 394ms
2:	learn: 0.6591197	total: 12.5ms	remaining: 366ms
3:	learn: 0.6489056	total: 16.3ms	remaining: 355ms
4:	learn: 0.6394214	total: 20.2ms	remaining: 348ms
5:	learn: 0.6295165	total: 24.2ms	remaining: 342ms
6:	learn: 0.6194457	total: 25.6ms	remaining: 308ms
7:	learn: 0.6105241	total: 29.3ms	remaining: 304ms
8:	learn: 0.6017105	total: 33.3ms	remaining: 304ms
9:	learn: 0.5933331	total: 37ms	remaining: 299ms
10:	learn: 0.5841446	total: 40.7ms	remaining: 296ms
11:	learn: 0.5762849	total: 44.3ms	remaining: 292ms
12:	learn: 0.5682086	total: 48.3ms	remaining: 290ms
13:	learn: 0.5602203	total: 52ms	remaining: 286ms
14:	learn: 0.5528362	total: 54.3ms	remaining: 275ms
15:	learn: 0.5461703	total: 58.2ms	remaining: 273ms
16:	learn: 0.5392012	total: 61.9ms	remaining: 269ms
17:	learn: 0.5329285	total: 65.6ms	remaining: 266ms
18:	learn: 0.5267278	total: 69.6ms	remaining: 264ms
19:	learn: 0.5198367	total

69:	learn: 0.3348406	total: 200ms	remaining: 60ms
70:	learn: 0.3332973	total: 203ms	remaining: 57.1ms
71:	learn: 0.3310499	total: 206ms	remaining: 54.3ms
72:	learn: 0.3291247	total: 209ms	remaining: 51.4ms
73:	learn: 0.3268869	total: 211ms	remaining: 48.5ms
74:	learn: 0.3250921	total: 214ms	remaining: 45.6ms
75:	learn: 0.3228877	total: 216ms	remaining: 42.7ms
76:	learn: 0.3207870	total: 219ms	remaining: 39.8ms
77:	learn: 0.3193101	total: 222ms	remaining: 37ms
78:	learn: 0.3177470	total: 226ms	remaining: 34.3ms
79:	learn: 0.3157841	total: 229ms	remaining: 31.4ms
80:	learn: 0.3140546	total: 231ms	remaining: 28.6ms
81:	learn: 0.3120020	total: 234ms	remaining: 25.7ms
82:	learn: 0.3100133	total: 237ms	remaining: 22.8ms
83:	learn: 0.3083180	total: 240ms	remaining: 20ms
84:	learn: 0.3067302	total: 244ms	remaining: 17.2ms
85:	learn: 0.3053659	total: 247ms	remaining: 14.3ms
86:	learn: 0.3040303	total: 249ms	remaining: 11.5ms
87:	learn: 0.3024998	total: 250ms	remaining: 8.54ms
88:	learn: 0.30079

In [ ]:
# apply Cat Boost Classifier with optuna.

In [46]:
def optimize(trial, x, y):
    learning_rate = trial.suggest_float('learning_rate',0.01,0.1)
    depth = trial.suggest_int('depth', 4, 16)
    iterations = trial.suggest_int('iterations', 50, 150)
    l2_leaf_reg = trial.suggest_int('l2_leaf_reg', 1, 5)
    model = catboost.CatBoostClassifier(
        learning_rate = learning_rate,
        depth = depth,
        iterations = iterations,
        l2_leaf_reg = l2_leaf_reg
    )

    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return np.mean(accuracies)

optimaization_function = partial(optimize,x = X, y = y)

study = optuna.create_study(direction = 'maximize')
study.optimize(optimaization_function, n_trials = 15)
print(study.best_params, study.best_value)

[I 2022-08-12 12:05:55,990] A new study created in memory with name: no-name-02f481f4-550a-4f4b-aecd-002305bdb5d8


0:	learn: 0.6365683	total: 768us	remaining: 84.5ms
1:	learn: 0.5926489	total: 1.83ms	remaining: 99.5ms
2:	learn: 0.5590120	total: 2.67ms	remaining: 96.2ms
3:	learn: 0.5289463	total: 3.56ms	remaining: 95.3ms
4:	learn: 0.5003161	total: 4.45ms	remaining: 94.4ms
5:	learn: 0.4722729	total: 5.36ms	remaining: 93.9ms
6:	learn: 0.4514530	total: 6.3ms	remaining: 93.6ms
7:	learn: 0.4300751	total: 7.2ms	remaining: 92.7ms
8:	learn: 0.4140132	total: 8.1ms	remaining: 91.7ms
9:	learn: 0.3994067	total: 8.96ms	remaining: 90.4ms
10:	learn: 0.3840876	total: 9.81ms	remaining: 89.2ms
11:	learn: 0.3700035	total: 10.7ms	remaining: 88.2ms
12:	learn: 0.3582213	total: 11.5ms	remaining: 87ms
13:	learn: 0.3482202	total: 12.4ms	remaining: 85.9ms
14:	learn: 0.3364043	total: 13.2ms	remaining: 84.6ms
15:	learn: 0.3273178	total: 14.4ms	remaining: 85.7ms
16:	learn: 0.3191138	total: 15.2ms	remaining: 84.3ms
17:	learn: 0.3110198	total: 16.1ms	remaining: 83.1ms
18:	learn: 0.3024465	total: 16.8ms	remaining: 81.5ms
19:	learn

0:	learn: 0.6399761	total: 2.77ms	remaining: 304ms
1:	learn: 0.5992621	total: 5.83ms	remaining: 318ms
2:	learn: 0.5622284	total: 8.39ms	remaining: 302ms
3:	learn: 0.5278928	total: 11.3ms	remaining: 304ms
4:	learn: 0.5002555	total: 14.1ms	remaining: 299ms
5:	learn: 0.4742131	total: 17.3ms	remaining: 303ms
6:	learn: 0.4534102	total: 19.9ms	remaining: 296ms
7:	learn: 0.4309007	total: 23.1ms	remaining: 298ms
8:	learn: 0.4136911	total: 26.1ms	remaining: 295ms
9:	learn: 0.3989874	total: 29ms	remaining: 293ms
10:	learn: 0.3831674	total: 31.8ms	remaining: 289ms
11:	learn: 0.3688281	total: 34.9ms	remaining: 288ms
12:	learn: 0.3568906	total: 38ms	remaining: 286ms
13:	learn: 0.3442120	total: 40.9ms	remaining: 283ms
14:	learn: 0.3317906	total: 43.8ms	remaining: 280ms
15:	learn: 0.3222679	total: 46.7ms	remaining: 277ms
16:	learn: 0.3127733	total: 49.6ms	remaining: 274ms
17:	learn: 0.3037738	total: 53.3ms	remaining: 275ms
18:	learn: 0.2955820	total: 56.4ms	remaining: 273ms
19:	learn: 0.2881188	total

0:	learn: 0.6392471	total: 1.75ms	remaining: 193ms
1:	learn: 0.5987689	total: 3.56ms	remaining: 194ms
2:	learn: 0.5615103	total: 5.38ms	remaining: 194ms
3:	learn: 0.5306166	total: 7.05ms	remaining: 189ms
4:	learn: 0.5025724	total: 8.68ms	remaining: 184ms
5:	learn: 0.4741232	total: 10.2ms	remaining: 179ms
6:	learn: 0.4532799	total: 12.1ms	remaining: 179ms
7:	learn: 0.4295734	total: 13.9ms	remaining: 178ms
8:	learn: 0.4095172	total: 15.6ms	remaining: 177ms
9:	learn: 0.3931887	total: 17.3ms	remaining: 175ms
10:	learn: 0.3771565	total: 19.1ms	remaining: 174ms
11:	learn: 0.3620634	total: 20.9ms	remaining: 172ms
12:	learn: 0.3486281	total: 22.6ms	remaining: 170ms
13:	learn: 0.3363337	total: 24.5ms	remaining: 170ms
14:	learn: 0.3254852	total: 26.2ms	remaining: 168ms
15:	learn: 0.3174457	total: 27.9ms	remaining: 166ms
16:	learn: 0.3081963	total: 29.6ms	remaining: 163ms
17:	learn: 0.2997596	total: 31.5ms	remaining: 163ms
18:	learn: 0.2919858	total: 33.2ms	remaining: 161ms
19:	learn: 0.2863428	t

[I 2022-08-12 12:05:57,825] Trial 0 finished with value: 0.9531932232082158 and parameters: {'learning_rate': 0.08994664221334535, 'depth': 4, 'iterations': 111, 'l2_leaf_reg': 1}. Best is trial 0 with value: 0.9531932232082158.


...

63:	learn: 0.1468201	total: 194ms	remaining: 237ms
64:	learn: 0.1462972	total: 196ms	remaining: 232ms
65:	learn: 0.1452911	total: 198ms	remaining: 228ms
66:	learn: 0.1448233	total: 200ms	remaining: 223ms
67:	learn: 0.1441046	total: 201ms	remaining: 219ms
68:	learn: 0.1434169	total: 203ms	remaining: 214ms
69:	learn: 0.1425759	total: 204ms	remaining: 210ms
70:	learn: 0.1416200	total: 206ms	remaining: 206ms
71:	learn: 0.1405460	total: 208ms	remaining: 202ms
72:	learn: 0.1398376	total: 209ms	remaining: 198ms
73:	learn: 0.1390710	total: 211ms	remaining: 194ms
74:	learn: 0.1380850	total: 213ms	remaining: 190ms
75:	learn: 0.1374272	total: 214ms	remaining: 186ms
76:	learn: 0.1362262	total: 216ms	remaining: 182ms
77:	learn: 0.1356262	total: 218ms	remaining: 178ms
78:	learn: 0.1349386	total: 220ms	remaining: 175ms
79:	learn: 0.1343780	total: 221ms	remaining: 171ms
80:	learn: 0.1335559	total: 223ms	remaining: 168ms
81:	learn: 0.1330559	total: 224ms	remaining: 164ms
82:	learn: 0.1320731	total: 226

0:	learn: 0.6361275	total: 5.88ms	remaining: 830ms
1:	learn: 0.5841921	total: 8.3ms	remaining: 581ms
2:	learn: 0.5438142	total: 10.9ms	remaining: 505ms
3:	learn: 0.5070804	total: 13.3ms	remaining: 458ms
4:	learn: 0.4742271	total: 15.8ms	remaining: 433ms
5:	learn: 0.4496119	total: 18.7ms	remaining: 424ms
6:	learn: 0.4285115	total: 21.8ms	remaining: 420ms
7:	learn: 0.4047221	total: 25ms	remaining: 419ms
8:	learn: 0.3818309	total: 28.7ms	remaining: 424ms
9:	learn: 0.3651106	total: 32.4ms	remaining: 427ms
10:	learn: 0.3495185	total: 37.2ms	remaining: 444ms
11:	learn: 0.3341438	total: 42.5ms	remaining: 461ms
12:	learn: 0.3195064	total: 47.9ms	remaining: 475ms
13:	learn: 0.3081633	total: 53.4ms	remaining: 488ms
14:	learn: 0.3014628	total: 59.2ms	remaining: 502ms
15:	learn: 0.2884825	total: 65.1ms	remaining: 512ms
16:	learn: 0.2802655	total: 70.9ms	remaining: 522ms
17:	learn: 0.2730714	total: 76.5ms	remaining: 527ms
18:	learn: 0.2655705	total: 81.9ms	remaining: 530ms
19:	learn: 0.2573943	tota

73:	learn: 0.1447299	total: 192ms	remaining: 177ms
74:	learn: 0.1439208	total: 196ms	remaining: 175ms
75:	learn: 0.1430429	total: 198ms	remaining: 172ms
76:	learn: 0.1421182	total: 200ms	remaining: 169ms
77:	learn: 0.1415763	total: 201ms	remaining: 165ms
78:	learn: 0.1403858	total: 203ms	remaining: 162ms
79:	learn: 0.1397035	total: 205ms	remaining: 159ms
80:	learn: 0.1391075	total: 207ms	remaining: 156ms
81:	learn: 0.1386256	total: 208ms	remaining: 152ms
82:	learn: 0.1384131	total: 210ms	remaining: 149ms
83:	learn: 0.1374932	total: 211ms	remaining: 146ms
84:	learn: 0.1374189	total: 213ms	remaining: 143ms
85:	learn: 0.1365242	total: 214ms	remaining: 140ms
86:	learn: 0.1357653	total: 216ms	remaining: 137ms
87:	learn: 0.1353614	total: 218ms	remaining: 134ms
88:	learn: 0.1345635	total: 220ms	remaining: 131ms
89:	learn: 0.1340976	total: 222ms	remaining: 128ms
90:	learn: 0.1330661	total: 224ms	remaining: 125ms
91:	learn: 0.1324091	total: 225ms	remaining: 122ms
92:	learn: 0.1315373	total: 227

[I 2022-08-12 12:14:13,466] Trial 14 finished with value: 0.9546974260617439 and parameters: {'learning_rate': 0.0994086433826426, 'depth': 7, 'iterations': 142, 'l2_leaf_reg': 2}. Best is trial 13 with value: 0.9549968259113687.


{'learning_rate': 0.07296679920960229, 'depth': 6, 'iterations': 133, 'l2_leaf_reg': 2} 0.9549968259113687


# Logistic Regression 

In [4]:
results = model_selection.cross_val_score(linear_model.LogisticRegression(),X,y,cv = model_selection.StratifiedKFold())
print(results)
print('Mean:',np.mean(results))


[0.85157421 0.85457271 0.85007496 0.85435435 0.84984985]
Mean: 0.8520852186519352


in Summary:

- the mean of accuracies for all algorithms was (between 85% and 86.5) except (Boosting Algorithms).
- boosting algorithms mean accuracies was (94% to 96% )Approximately.
- at most all hyper parameter tuning techniques with the same algorithm not have big differace values for              accuracies.
- I have used Stratified Shuffle Split for cross validation score becuse targer values are inbalanced.
- I prefer to use aptuna ,hyperopt or same times gp_minimize more than gradesearch and randomizesdsearch                becuse the time complexty for the last tow is very very big.  
- for this dataset i will chose XGB as Classifier with hyperopt as hyper parameter tuning thats give 95%            accuracy becuse its not take that time like Gradient Boost ot Cat Boost.

# Final Model :

In [19]:
# model = model_selection.cross_val_score(xgb.XGBClassifier(eta = 0.10374145404246778, max_depth = 4, sampling_method = 'uniform'),
#                                                X,y,cv = model_selection.StratifiedShuffleSplit())
# print(np.mean(model))
# print(metrics.confusion_matrix(y, model))

model = xgb.XGBClassifier(eta = 0.10374145404246778, max_depth = 4, sampling_method = 'uniform')
sss = model_selection.StratifiedShuffleSplit()
acc_list = []
for idx in sss.split(X = X, y = y):
    
    xtrain, ytrain = X[idx[0]], y[idx[0]] 
    xtest, ytest = X[idx[1]], y[idx[1]]
    
    model.fit(xtrain, ytrain)
    y_pred = model.predict(xtest)
    print(metrics.confusion_matrix(ytest,y_pred))
    print(metrics.accuracy_score(ytest, y_pred))
    acc_list.append(metrics.accuracy_score(ytest, y_pred))
    print(metrics.classification_report(ytest,y_pred))
    print('--------------------------------------------------------')
print('Mean:',np.mean(acc_list))

[[281   5]
 [ 12  36]]
0.9491017964071856
              precision    recall  f1-score   support

           0       0.96      0.98      0.97       286
           1       0.88      0.75      0.81        48

    accuracy                           0.95       334
   macro avg       0.92      0.87      0.89       334
weighted avg       0.95      0.95      0.95       334

--------------------------------------------------------
[[284   2]
 [ 11  37]]
0.9610778443113772
              precision    recall  f1-score   support

           0       0.96      0.99      0.98       286
           1       0.95      0.77      0.85        48

    accuracy                           0.96       334
   macro avg       0.96      0.88      0.91       334
weighted avg       0.96      0.96      0.96       334

--------------------------------------------------------
[[284   2]
 [ 11  37]]
0.9610778443113772
              precision    recall  f1-score   support

           0       0.96      0.99      0.98       2